In [4]:
import pandas as pd

# Load the CSV
df = pd.read_csv("results_handcrafted.csv")

# Ensure findings is string type
df['findings'] = df['findings'].astype(str)

# Define the variations you want to match
reentrancy_variants = ["Reentrancy", "Re_Entrancy", "reentrancy", "REENTRANCY", "State_access_after_external_call_SWC_107"]

# Check if any variant is in the findings string
df['detected_reentrancy'] = df['findings'].apply(
    lambda x: any(variant in x for variant in reentrancy_variants)
)


# Filter rows where reentrancy was detected
reentrancy_df = df[df['detected_reentrancy']]

# Group by basename and aggregate tool names
reentrancy_summary = (
    reentrancy_df
    .groupby('basename')
    .agg(
        tools_detected_reentrancy=('toolid', lambda x: sorted(set(x))),
        num_tools_detected_reentrancy=('toolid', lambda x: x.nunique())
    )
    .reset_index()
)

# Optional: include all contracts and fill 0/tools if no detection
all_contracts = df[['basename']].drop_duplicates()
result = all_contracts.merge(reentrancy_summary, on='basename', how='left')
result['num_tools_detected_reentrancy'] = result['num_tools_detected_reentrancy'].fillna(0).astype(int)
result['tools_detected_reentrancy'] = result['tools_detected_reentrancy'].apply(lambda x: x if isinstance(x, list) else [])

# Print the result
result


,basename,tools_detected_reentrancy,num_tools_detected_reentrancy
0,00_BasicConst_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4]",3
1,00_BasicConst_safe1.sol,[],0
2,00_BasicEmit_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4]",3
3,00_BasicEmit_safe1.sol,"[confuzzius, slither-0.10.4]",2
4,00_BasicError_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4]",3
...,...,...,...
121,17_SidefxInline_ree1.sol,"[confuzzius, mythril-0.24.7]",2
122,18_Rare_ree2.sol,[slither-0.10.4],1
123,18_Rare_safe1.sol,[],0
124,18_Rare_safe2.sol,[slither-0.10.4],1


In [5]:
pd.set_option('display.max_rows', None)  # Show all rows
result = result.sort_values(by='num_tools_detected_reentrancy', ascending=False)
result

,basename,tools_detected_reentrancy,num_tools_detected_reentrancy
60,07_MixedTransfer_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4, s...",4
78,09_ERC20DD_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4, s...",4
66,08_MixedSend_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4, s...",4
68,09_ERC20DDMod_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4, s...",4
64,08_MixedSendFold_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4, s...",4
62,08_MixedSendFoldEmit_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4, s...",4
12,00_BasicHuman_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4]",3
13,00_BasicHuman_ree2.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4]",3
10,00_BasicHumanFold_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4]",3
0,00_BasicConst_ree1.sol,"[confuzzius, mythril-0.24.7, slither-0.10.4]",3
